### Import Modules & Input Dataset

In [1]:
import pandas as pd
# read extra data and change date into date format
df = pd.read_csv('../data/raw/extra_data.csv')
df['date'] = pd.to_datetime(df['date'])


### Data Preprocessing

In [2]:
# Generate dataframe with all days of 2022
all_dates = pd.date_range(start='2022-01-01', end='2022-12-31')
df_all = pd.DataFrame(all_dates, columns=['date'])
df_all['event'] = 'workingday'  # give all day with "workingday" first
df_combined = pd.concat([df, df_all], ignore_index=True)

# Change when date == weekend, value in event from "workingday" to "weekend"
df_combined.loc[(df_combined['date'].dt.dayofweek >= 5) & (df_combined['event'] == 'workingday'), 'event'] = 'weekend'

# Order by date, form 01/01/2022 to 31/12/2022
df_combined = df_combined.drop_duplicates(subset=['date'], keep='first')
df_combined = df_combined.sort_values(by='date')

# Calculate unique values count in 'event' column and print
unique_counts = df_combined['event'].value_counts()
print("2022 Leuven Calendar Counts:")
print(unique_counts)
print(f"Total days: {unique_counts.sum()}")

2022 Leuven Calendar Counts:
event
workingday        134
exam               97
weekend            50
summerholidays     44
easter             17
winterholidays      8
christmas           8
labour              1
ascension           1
natonalday          1
opening             1
allsaints           1
allsouls            1
armisticeday        1
Name: count, dtype: int64
Total days: 365


In [3]:
# Duplicate 24 times for every throw and add hours
df_combined = df_combined.loc[df_combined.index.repeat(24)].reset_index(drop=True)
df_combined['hour'] = df_combined.groupby(df_combined['date'].dt.date).cumcount()
df_combined['date'] = df_combined['date'] + pd.to_timedelta(df_combined['hour'], unit='h')

# Generate new col "event2" with all bankholidays and new col "break"
events_to_replace = ['natonalday', 'opening', 'allsaints', 'allsouls', 'armisticeday','ascension','labour']
df_combined['event2'] = df_combined['event'].apply(lambda x: 'bankholidays' if x in events_to_replace else x)
df_combined['break'] = df_combined['event'].apply(lambda x: 0 if x in ['exam', 'workingday'] else 1)

# One-hot encode for "event2" and rename
df_combined = pd.get_dummies(df_combined, columns=['event2'])
for col in df_combined.columns[df_combined.columns.str.startswith('event2')]:
    df_combined[col] = df_combined[col].astype(int)
df_combined.columns = df_combined.columns.str.replace('event2_', 'if_')
df_combined = df_combined.rename(columns={'if_workingday': 'if_workingday','if_bankholidays':'if_bankholiday',
                                          'if_summerholidays':'if_summerholiday',
                                          'if_winterholidays':'if_winterholiday'})

# Select only 'date' and break' columns for model training data
df_for_model = df_combined[['date', 'break']]
df_for_model.to_csv('model_event.csv', index=False)

# Drop 'event', 'hour' and 'event2', save other columns for DEA data
df_combined = df_combined.drop(columns=['event', 'break', 'hour'])
df_combined.to_csv('event_onehote.csv', index=False)

# Print head of df_combined
df_combined.head()

,date,if_bankholiday,if_christmas,if_easter,if_exam,if_summerholiday,if_weekend,if_winterholiday,if_workingday
0,2022-01-01 00:00:00,0,0,0,1,0,0,0,0
1,2022-01-01 01:00:00,0,0,0,1,0,0,0,0
2,2022-01-01 02:00:00,0,0,0,1,0,0,0,0
3,2022-01-01 03:00:00,0,0,0,1,0,0,0,0
4,2022-01-01 04:00:00,0,0,0,1,0,0,0,0
